In [1]:
# ==========================================
# Sales Data Preprocessing and Preparation
# ==========================================

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, Normalizer

# Load the dataset
file_path = "C:\\Users\\ranji\\Documents\\DELL\\Data Analytics & Data Science\\18. 1st June\\sales.csv"
df = pd.read_csv("C:\\Users\\ranji\\Documents\\DELL\\Data Analytics & Data Science\\18. 1st June\\sales.csv", encoding='latin1')

# Select only useful columns for modeling
df = df[['QUANTITYORDERED', 'PRICEEACH', 'STATUS', 'PRODUCTLINE', 'DEALSIZE', 'SALES']]

print("Initial Data:")
print(df.head(), "\n")

# -----------------------------
# Handle missing values
# -----------------------------
df['QUANTITYORDERED'].fillna(df['QUANTITYORDERED'].mean(), inplace=True)
df['PRICEEACH'].fillna(df['PRICEEACH'].median(), inplace=True)
df['STATUS'].fillna('Unknown', inplace=True)
df['PRODUCTLINE'].fillna('Unknown', inplace=True)
df['DEALSIZE'].fillna('Unknown', inplace=True)
df['SALES'].fillna(df['SALES'].median(), inplace=True)

print("After Filling Missing Values:")
print(df.isnull().sum(), "\n")

# -----------------------------
# Encode categorical variables
# -----------------------------
# One-hot encode categorical columns
df = pd.get_dummies(df, columns=['STATUS', 'PRODUCTLINE', 'DEALSIZE'], drop_first=True)
print("After Encoding Categorical Variables:")
print(df.head(), "\n")

# -----------------------------
# Handle outliers (IQR Method)
# -----------------------------
columns_with_outliers = ['QUANTITYORDERED', 'PRICEEACH', 'SALES']

for col in columns_with_outliers:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outlier_indices = np.where((df[col] < lower_bound) | (df[col] > upper_bound))
    print(f"Outliers Detected in '{col}' at Indices: {outlier_indices[0]}")

    if len(outlier_indices[0]) > 0:
        df.loc[outlier_indices[0], col] = df[col].median()
        print(f"Outliers in column '{col}' replaced with the median.\n")
    else:
        print(f"No outliers detected in '{col}'.\n")

print("After Outlier Handling:")
print(df.describe(), "\n")

# -----------------------------
# Correlation Check
# -----------------------------
print("Correlation Matrix:")
print(df.corr(), "\n")

# -----------------------------
# Split the data
# -----------------------------
X = df.drop(['SALES'], axis=1)
y = df['SALES']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training Features (X_train):")
print(X_train.head(), "\n")
print("Training Labels (y_train):")
print(y_train.head(), "\n")

# -----------------------------
# Scaling and Normalization
# -----------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
print("Scaled Training Data (first 5 rows):\n", X_train_scaled[:5], "\n")

normalizer = Normalizer()
X_train_normalized = normalizer.fit_transform(X_train)
print("Normalized Training Data (first 5 rows):\n", X_train_normalized[:5])

print("\n✅ Data preprocessing completed successfully! Ready for regression modeling.")


Initial Data:
   QUANTITYORDERED  PRICEEACH   STATUS  PRODUCTLINE DEALSIZE    SALES
0               30      95.70  Shipped  Motorcycles    Small  2871.00
1               34      81.35  Shipped  Motorcycles    Small  2765.90
2               41      94.74  Shipped  Motorcycles   Medium  3884.34
3               45      83.26  Shipped  Motorcycles   Medium  3746.70
4               49     100.00  Shipped  Motorcycles   Medium  5205.27 

After Filling Missing Values:
QUANTITYORDERED    0
PRICEEACH          0
STATUS             0
PRODUCTLINE        0
DEALSIZE           0
SALES              0
dtype: int64 

After Encoding Categorical Variables:
   QUANTITYORDERED  PRICEEACH    SALES  STATUS_Disputed  STATUS_In Process  \
0               30      95.70  2871.00            False              False   
1               34      81.35  2765.90            False              False   
2               41      94.74  3884.34            False              False   
3               45      83.26  3746.70     